# UAT-C5-001: Simple and Intuitive User Configuration

In [ ]:
from mlip_autopipec.config.models import UserInputConfig
from mlip_autopipec.workflow_manager import WorkflowManager
import yaml

user_yaml = """
project_name: "My FeNi Alloy Study"
target_system:
  elements: ["Fe", "Ni"]
  composition: { "Fe": 0.7, "Ni": 0.3 }
  crystal_structure: "fcc"
simulation_goal:
  type: "melt_quench"
  temperature_range: [300, 2000]
"""

print("--- This is all you need to provide: ---")
print(user_yaml)

user_config_dict = yaml.safe_load(user_yaml)
user_config = UserInputConfig.model_validate(user_config_dict)

manager = WorkflowManager(user_config)

print("\n--- The System's Expanded View (Highlights): ---")
print(f"Project Name: {manager.system_config.project_name}")
print(f"Run UUID: {manager.system_config.run_uuid}")
print(f"Fingerprint Species: {manager.system_config.explorer_config.fingerprint.species}")
print(f"MD Temperature: {manager.system_config.inference_config.md_params.temperature} K")
print(
    f"Magnetism Enabled: {manager.system_config.dft_config.dft_input_params.magnetism is not None}"
)

# UAT-C5-002: Robust Validation of User Input

In [ ]:
from pydantic import ValidationError

invalid_yaml_sum = """
project_name: "Test Project"
target_system:
  elements: ["Fe", "Ni"]
  composition: { "Fe": 0.7, "Ni": 0.2 }
  crystal_structure: "fcc"
simulation_goal: { type: "melt_quench" }
"""

print("--- Testing Composition Sum Error: ---")
try:
    UserInputConfig.model_validate(yaml.safe_load(invalid_yaml_sum))
except ValidationError as e:
    print(f"Successfully caught error: {e}")

invalid_yaml_element = """
project_name: "Test Project"
target_system:
  elements: ["Fe", "Nx"]
  composition: { "Fe": 0.5, "Nx": 0.5 }
  crystal_structure: "fcc"
simulation_goal: { type: "melt_quench" }
"""

print("\n--- Testing Invalid Element Error: ---")
try:
    UserInputConfig.model_validate(yaml.safe_load(invalid_yaml_element))
except ValidationError as e:
    print(f"Successfully caught error: {e}")

# UAT-C5-003: Verification of Rich Metadata in Database

In [ ]:
from pathlib import Path
import numpy as np
from ase import Atoms
from ase.db import connect
from uuid import uuid4
from mlip_autopipec.config.models import DFTResult
from mlip_autopipec.utils.ase_utils import save_dft_result

db_path = Path("uat_c5_test.db")
if db_path.exists():
    db_path.unlink()

atoms = Atoms("H2", positions=[[0, 0, 0], [0, 0, 1]])
result = DFTResult(
    job_id=uuid4(), energy=-1.0, forces=[[0, 0, 0], [0, 0, 0]], stress=[0, 0, 0, 0, 0, 0]
)
metadata = {
    "uuid": str(result.job_id),
    "config_type": "active_learning_gen_4",
    "force_mask": np.array([1, 0]),
}

save_dft_result(db_path, atoms, result, metadata)

print("--- Verifying data from database: ---")
with connect(db_path) as db:
    row = db.get(1)
    retrieved_atoms = row.toatoms()
    print(f"Config Type: {row.get('config_type')}")
    print(f"UUID: {row.get('uuid')}")
    print(f"Force Mask: {retrieved_atoms.arrays['force_mask']}")
    assert row.get("config_type") == "active_learning_gen_4"
    assert np.array_equal(retrieved_atoms.arrays["force_mask"], np.array([1, 0]))

db_path.unlink()